<a href="https://colab.research.google.com/github/sanjayy0612/stock_predictor-NVDIA-/blob/main/NVDIA_STOCK_PREDICTOR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install yfinance


In [1]:
import yfinance as yf
import pandas as pd


nvda = yf.download("NVDA", start="2022-01-01", end="2025-09-11", interval='1d')

/tmp/ipython-input-3776370923.py:5: FutureWarning: YF.download() has changed argument auto_adjust default to True
  nvda = yf.download("NVDA", start="2022-01-01", end="2025-09-11", interval='1d')
[*********************100%***********************]  1 of 1 completed


In [2]:

if isinstance(nvda.columns, pd.MultiIndex):
    nvda.columns = [col[0] for col in nvda.columns]

nvda["Return"] = nvda["Close"].pct_change()
nvda["MA10"] = nvda["Close"].rolling(window=10).mean()
nvda["MA50"] = nvda["Close"].rolling(window=50).mean()
nvda["Volatility"] = nvda["Return"].rolling(window=10).std()
nvda = nvda.dropna()
print(nvda.head(15))
print("\nAvailable columns:", nvda.columns)


                Close       High        Low       Open     Volume    Return  \
Date                                                                          
2022-03-15  22.935061  22.999954  21.286788  21.482463  491996000  0.077028   
2022-03-16  24.455544  24.556377  23.133731  23.461188  671422000  0.066295   
2022-03-17  24.725100  24.800972  23.866519  24.080167  471941000  0.011022   
2022-03-18  26.409311  26.525121  24.583333  24.758044  730719000  0.068117   
2022-03-21  26.689854  27.107165  25.924120  26.463229  591727000  0.010623   
2022-03-22  26.480196  27.193018  26.028943  26.682860  547007000 -0.007855   
2022-03-23  25.591669  26.568053  25.532766  26.082855  502120000 -0.033554   
2022-03-24  28.103512  28.273231  25.864216  26.122789  877379000  0.098151   
2022-03-25  27.646271  28.311171  27.225966  27.839951  579016000 -0.016270   
2022-03-28  28.172398  28.203347  27.161070  27.710163  425494000  0.019031   
2022-03-29  28.608677  28.898197  27.933793  28.6476

In [3]:
nvda["Target_UpDown"] = (nvda["Close"].shift(-1) > nvda["Close"]).astype(int)
nvda["Target_Close"] = nvda["Close"].shift(-1)
nvda.dropna()

,Close,High,Low,Open,Volume,Return,MA10,MA50,Volatility,Target_UpDown,Target_Close
Date,,,,,,,,,,,
2022-03-15,22.935061,22.999954,21.286788,21.482463,491996000,0.077028,22.543810,24.608723,0.047368,1,24.455544
2022-03-16,24.455544,24.556377,23.133731,23.461188,671422000,0.066295,22.571364,24.496511,0.051139,1,24.725100
2022-03-17,24.725100,24.800972,23.866519,24.080167,471941000,0.011022,22.676390,24.406280,0.050523,1,26.409311
2022-03-18,26.409311,26.525121,24.583333,24.758044,730719000,0.068117,23.027509,24.383392,0.052080,1,26.689854
2022-03-21,26.689854,27.107165,25.924120,26.463229,591727000,0.010623,23.564820,24.354655,0.042995,0,26.480196
...,...,...,...,...,...,...,...,...,...,...,...
2025-09-03,170.619995,172.410004,168.880005,171.059998,164424900,-0.000937,176.729999,171.589600,0.014903,1,171.660004
2025-09-04,171.660004,171.860001,169.410004,170.570007,141670100,0.006095,176.356000,172.064800,0.015167,0,167.020004
2025-09-05,167.020004,169.029999,164.070007,168.029999,224441400,-0.027030,175.560001,172.319000,0.017106,1,168.309998


In [4]:
from sklearn.model_selection import train_test_split

X = nvda[["Return", "MA10", "MA50", "Volatility"]]
y = nvda[["Target_UpDown", "Target_Close"]]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, shuffle=False
)

y_class_train = y_train["Target_UpDown"]
y_class_test  = y_test["Target_UpDown"]

y_reg_train   = y_train["Target_Close"]
y_reg_test    = y_test["Target_Close"]

In [ ]:
print(nvda["Target_UpDown"].value_counts())


Target_UpDown
1    470
0    397
Name: count, dtype: int64


## THE CLASSIFICATION MODEL

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix


clf = LogisticRegression(max_iter=1000)
clf.fit(X_train, y_class_train)


y_class_pred = clf.predict(X_test)


print("Classification Accuracy:", accuracy_score(y_class_test, y_class_pred))
print("\nClassification Report:\n", classification_report(y_class_test, y_class_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_class_test, y_class_pred))



Classification Accuracy: 0.5517241379310345

Classification Report:
               precision    recall  f1-score   support

           0       0.00      0.00      0.00        78
           1       0.55      1.00      0.71        96

    accuracy                           0.55       174
   macro avg       0.28      0.50      0.36       174
weighted avg       0.30      0.55      0.39       174


Confusion Matrix:
 [[ 0 78]
 [ 0 96]]


/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


The model isn’t predicting well because the dataset and features don’t give  clear patterns to separate Up vs Down days. Since there are slightly more Up days (470 vs 397), the logistic regression model falls into a “lazy” solution: it always predicts Up to maximize accuracy without actually learning meaningful trends.

That’s why the confusion matrix is:

[[ 0 78]  
 [ 0 96]]


It shows the model never predicted a Down (0). In other words, it learned a trivial rule (“always Up”) instead of understanding the stock’s real movements.




## RANDOM FOREST

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf_clf = RandomForestClassifier(n_estimators=100, random_state=42)
rf_clf.fit(X_train, y_class_train)
print("Training complete.")

y_rf_pred = rf_clf.predict(X_test)
print("Classification Accuracy:", accuracy_score(y_class_test, y_rf_pred))
print("\nClassification Report:\n", classification_report(y_class_test, y_rf_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_class_test, y_rf_pred))



Training complete.
Classification Accuracy: 0.5172413793103449

Classification Report:
               precision    recall  f1-score   support

           0       0.47      0.62      0.53        78
           1       0.58      0.44      0.50        96

    accuracy                           0.52       174
   macro avg       0.53      0.53      0.52       174
weighted avg       0.53      0.52      0.51       174


Confusion Matrix:
 [[48 30]
 [54 42]]


Even though the accuracy of Logistic Regression and Random Forest is quite similar (~51–52%), there’s an important difference in how they make predictions. Logistic Regression was “lazy” in the sense that it leaned heavily towards predicting the stock would always go up, which inflated its accuracy but didn’t really capture the true movement of the market.

Random Forest, on the other hand, provided a more balanced perspective. It not only predicted both up and down days but also showed better distribution in its precision and recall. While the overall accuracy didn’t skyrocket, the model demonstrated that it could actually detect down days with a higher recall (62%), something Logistic Regression struggled with.

This shows that accuracy alone isn’t always the best measure of performance—what really matters is whether the model captures the underlying patterns in the data. In this case, Random Forest gave more meaningful and reliable predictions, making it the stronger choice for this kind of stock movement classification.

# GRIDSEARCH IN RANDOMFOREST

Trying GRIDSEARCHcv in the RandomForest classifier so that the model would predict much then before


An optimized technique to find the best Hyperparemeters for the model which is done after doing the training










In [8]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.ensemble import RandomForestClassifier

rfc=RandomForestClassifier(random_state=42)

param_grid = {
    'n_estimators': [100, 200, 500],
    'max_depth': [None, 5, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}
grid_search=GridSearchCV(
    estimator=rfc,
    param_grid=param_grid,
    verbose=3,
    n_jobs=-1,
    cv=5
    )
grid_search.fit(X_train, y_class_train)
print(grid_search.best_params_)

best_est = grid_search.best_estimator_
y_pred = best_est.predict(X_test)
print(accuracy_score(y_class_test, y_pred))



Fitting 5 folds for each of 108 candidates, totalling 540 fits
{'max_depth': 5, 'min_samples_leaf': 4, 'min_samples_split': 10, 'n_estimators': 200}
0.5
